In [1]:
# Numerical/Stats pack
import numpy as np
from collections import Counter
import sys
import math

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


In [2]:
def get_basis(num_qubits, index):
    """
    Parameters
    ----------
    num_qubits : int
        number of qubits.
    index : int
        Which qubit is 1. Order follows measurement order

    Returns
    -------
    base : int
        the basis string where target position is 1
        e.g., num_qubits=5, index=1, then return '01000'

    """
    origin = list('0'*num_qubits)
    origin[index] = '1'
    return ''.join(origin)
    
def qubit_matrix(p0m0,p1m1):
    """
    Parameters
    ----------
    p0m0 : ndarray(dtype=np.float64, dim=num_qubits)
        Pr(0|0) for all qubits
    p1m1 : ndarray(dtype=np.float64, dim=num_qubits)
        Pr(1|1) for all qubits

    Returns
    -------
    mats: np.array(dim=num_qubits*2*2)
        mats[i] is 2-by-2 measurement error transition matrix for ith qubit
    """
    num_qubits = p0m0.shape[0]
    mats = np.zeros((num_qubits,2,2), dtype=np.float64)
    for i in range(num_qubits):
        mats[i] = np.array([[p0m0[i], 1-p1m1[i]],[1-p0m0[i], p1m1[i]]])
        
    return mats

def dictToVec(nQubits, counts):
    """ Transfer counts to probabilities

    Args:
      nQUbits: int
        number of qubits
      counts: dict
        an dictionary in the form {basis string: frequency}. E.g.
        {"01": 100
         "11": 100}
        dict key follow little-endian convention

    Returns: numpy array
      an probability vector (array). E.g.
      [0, 0.5, 0, 0.5] is the result from example above.
    """
    vec = np.zeros(2**nQubits)
    form = "{0:0" + str(nQubits) + "b}"
    for i in range(2**nQubits):
        key = form.format(i) # consider key = format(i,'0{}b'.format(nQubits))
                             # and delete variable form
        if key in counts.keys():
            vec[i] = int(counts[key])
        else:
            vec[i] = 0
    return vec


def print_info(mat):
    print("Largest entry={:.4g}, Frobenius norm={:.4g}, 2-Norm={:.4g}".format(np.abs(mat).max(), 
                                                                              np.linalg.norm(mat, ord='fro'),
                                                                              np.linalg.norm(mat, ord=2)))
    print("Among {:d} entries, {:d} entries < 1e-3, {:d} entries < 1e-4, {:d} entries < 1e-6".format(mat.shape[0]*mat.shape[1],
                                                                                                      mat[np.abs(mat)<1e-3].shape[0],
                                                                                                      mat[np.abs(mat)<1e-4].shape[0], 
                                                                                                      mat[np.abs(mat)<1e-6].shape[0]))

In [3]:
# Cite https://www.geeksforgeeks.org/check-whether-two-numbers-differ-one-bit-position/
# Python3 implementation to check whether the two  
# numbers differ at one bit position only 
  
# function to check if x is power of 2 
def isPowerOfTwo( x ): 
  
    # First x in the below expression is 
    # for the case when x is 0 
    return x and (not(x & (x - 1))) 
  
# function to check whether the two numbers 
# differ at one bit position only 
def differAtOneBitPos( a , b ): 
    return isPowerOfTwo(a ^ b) 
 
# Function to find the position of 
# rightmost set bit in 'n'
def getRightMostSetBit(n):
    if (n == 0):
        return 0
 
    return math.log2(n & -n)
 
# Cite https://www.geeksforgeeks.org/position-rightmost-different-bit/
# Function to find the position of 
# rightmost different bit in the 
# binary representations of 'm' and 'n'
def posOfRightMostDiffBit(m, n):
 
    # position of rightmost different
    # bit
    return int(getRightMostSetBit(m ^ n))

In [4]:
# Parameters
file_address = "./TestData/"
num_qubits = 2

# Method 1

For example, if we have 5 qubits and we compute Pr(0|0) for Qubit 1, then we go to **data from circuit that constructs 00000** and have $$ Pr(0|0) =1 - \frac{m(01000)}{m(00000) + m(01000)}, $$ where $m()$ is the number of appearences of a given base in some dataset.

Similarly, if we compute Pr(1|1) for Qubit 1, then we go to **data from circuit that constructs 11111** and have $$ Pr(1|1) = 1 - \frac{m(10111)}{m(10111) + m(11111)}. $$

So we need 2 circuits.



## Lima

Difference between full matrix and the matrix from this method:

Largest entry=0.2287, Frobenius norm=1.158, 2-Norm=0.344

Among 1024 entries, 734 entries < 1e-3, 567 entries < 1e-4, 348 entries < 1e-6

## Quito

Largest entry=0.1525, Frobenius norm=0.7954, 2-Norm=0.2537

Among 1024 entries, 745 entries < 1e-3, 525 entries < 1e-4, 328 entries < 1e-6

In [5]:
# p0m1_count = np.zeros(num_qubits, dtype=np.int) # Record number of appearease of prepare 0 measure 1
# p0m0_total = np.zeros(num_qubits, dtype=np.int) # Record number of data entries does not have correlated errors

# string_data = np.genfromtxt(file_address + 'Basis'+('0'*num_qubits)+'.csv', delimiter=',',dtype=np.str)
# for bits in string_data:
#     if bits.count("1") == 0: # When all qubits are noise-free
#         p0m0_total = p0m0_total + 1 
#     if bits.count("1") == 1:
#         pos = string_data[2].find('1') # Postion of the bit 1
#         p0m1_count[pos] += 1
#         p0m0_total[pos] += 1

# # Record probability of prepare 0 measure 0
# p0m0 = (p0m0_total-p0m1_count)/p0m0_total


# p1m0_count = np.zeros(num_qubits, dtype=np.int) # Record number of appearease of prepare 1 measure 0
# p1m1_total = np.zeros(num_qubits, dtype=np.int) # Record number of data entries does not have correlated errors

# string_data = np.genfromtxt(file_address + 'Basis'+('1'*num_qubits)+'.csv', delimiter=',',dtype=np.str)
# for bits in string_data:
#     if bits.count("0") == 0: # When all qubits are noise-free
#         p1m1_total = p1m1_total + 1 
#     if bits.count("0") == 1:
#         pos = string_data[2].find('0') # Postion of the bit 1
#         p1m0_count[pos] += 1
#         p1m1_total[pos] += 1

# # Record probability of prepare 1 measure 1
# p1m1 = (p1m1_total-p1m0_count)/p1m1_total

# Method 2

For example, if we have 5 qubits and we compute Pr(0|0) for Qubit 1, then we go to **data from circuit that constructs 10111** and have $$ Pr(0|0) = \frac{m(10111)}{m(10111) + m(11111)}, $$ where $m()$ is the number of appearences of a given base in some dataset.

Similarly, if we compute Pr(1|1) for Qubit 1, then we go to **data from circuit that constructs 01000** and have $$ Pr(1|1) = \frac{m(01000)}{m(01000) + m(00000)}. $$

So we need $2n$ number of circuits where $n$ is the number of qubits.


 ## Lima

Difference between full matrix and the matrix from this method:

Largest entry=0.005453, Frobenius norm=0.01927, 2-Norm=0.007424

Among 1024 entries, 956 entries < 1e-3, 762 entries < 1e-4, 227 entries < 1e-6


## Quito
Largest entry=0.008537, Frobenius norm=0.0245, 2-Norm=0.01069

Among 1024 entries, 942 entries < 1e-3, 739 entries < 1e-4, 203 entries < 1e-6

In [11]:
p0m0 = np.zeros(num_qubits) # Record probability of prepare 0 measure 0
p0m0_total = np.zeros(num_qubits) # Record number of data entries does not have correlated errors


for index in range(num_qubits):
    # Read data
    temp_string = get_basis(num_qubits, index)
    target = ''.join('1' if s == '0' else '0' for s in temp_string) # Corresponding bit string
    string_data = np.genfromtxt(file_address + 'Basis'+target+'.csv', delimiter=',',dtype=np.str)
    # Record number of 1's when other qubits is noise-free
    num_data = 0
    num_0 = 0
    for bits in string_data:
        others = bits[:index]+bits[index+1:] # The bits that don't include the interested qubits
        if others == '1'*(num_qubits-1): # Make sure other qubits is noise-free
            num_data += 1
            if bits[index] == '0':
                num_0 += 1
                
    # Record data
    p0m0[index] = num_0/num_data
    p0m0_total[index] = num_data
    
    
p1m1 = np.zeros(num_qubits) # Record probability of prepare 1 measure 1
p1m1_total = np.zeros(num_qubits) # Record number of data entries does not have correlated errors


for index in range(num_qubits):
    # Read data
    target = get_basis(num_qubits, index) # Corresponding bit string
    string_data = np.genfromtxt(file_address + 'Basis'+target+'.csv', delimiter=',',dtype=np.str)
    # Record number of 1's when other qubits is noise-free
    num_data = 0
    num_1 = 0
    for bits in string_data:
        others = bits[:index]+bits[index+1:] # The bits that don't include the interested qubits
        if others == '0'*(num_qubits-1): # Make sure other qubits is noise-free
            num_data += 1
            if bits[index] == '1':
                num_1 += 1
                
    # Record data
    p1m1[index] = num_1/num_data
    p1m1_total[index] = num_data

# Method 3

Use all possible basis circuits

So we need $2^n$ number of circuits where $n$ is the number of qubits.

 
## Lima
Difference between full matrix and the matrix from this method:

Largest entry=0.004626, Frobenius norm=0.01495, 2-Norm=0.005593

Among 1024 entries, 979 entries < 1e-3, 764 entries < 1e-4, 220 entries < 1e-6

## Quito
Largest entry=0.006458, Frobenius norm=0.01722, 2-Norm=0.008856

Among 1024 entries, 965 entries < 1e-3, 727 entries < 1e-4, 191 entries < 1e-6

In [13]:
p0m0_count = np.zeros(num_qubits) # Record counts of prepare 0 measure 0
p0m0_total = np.zeros(num_qubits) # Record number of data entries does not have correlated errors
p1m1_count = np.zeros(num_qubits) # Record counts of prepare 1 measure 1
p1m1_total = np.zeros(num_qubits) # Record number of data entries does not have correlated errors

# Go through all possibile basis datasets
for bn in range(2**num_qubits):
    target = format(bn, "0{:d}b".format(num_qubits))
    string_data = np.genfromtxt(file_address + 'Basis'+target+'.csv', delimiter=',',dtype=np.str)
    
    # Then for each data (bitstring) in the dataset
    for meas_res in string_data:
        meas_int = int(meas_res, 2) # to binary number
        if meas_res == target: # If there is no error
            for i in range(num_qubits):
                if int(target[i],2): # if bit is 1
                    p1m1_count[i] += 1
                    p1m1_total[i] += 1
                else:
                    p0m0_count[i] += 1
                    p0m0_total[i] += 1
        
        if differAtOneBitPos(bn, meas_int): # if given basis and measured basis only differ by 1 bit
            diff_pos = num_qubits - posOfRightMostDiffBit(bn, meas_int) - 1 # position that the bit is different
            if target[diff_pos] == '0':
                p0m0_total[diff_pos] += 1
            else:
                p1m1_total[diff_pos] += 1
            
# Record probability of prepare 0 measure 0
p0m0 = p0m0_count/p0m0_total
# Record probability of prepare 1 measure 1
p1m1 = p1m1_count/p1m1_total

In [14]:
p0m0, p0m0_total, p1m1, p1m1_total

(array([0.83333333, 0.83333333]),
 array([12., 12.]),
 array([0.5, 0.5]),
 array([8., 8.]))

In [ ]:
np.sqrt(1/60000), np.sqrt(1/900000)

In [ ]:
single_qubit_mats = qubit_matrix(p0m0,p1m1)

In [ ]:
single_qubit_mats

In [ ]:
# matrix with independent assumption
ind_mat = single_qubit_mats[0]
for i in range(1,num_qubits):
    ind_mat = np.kron(ind_mat, single_qubit_mats[i])

In [ ]:
# Original Matrix
mean_mat = np.zeros((2**num_qubits,2**num_qubits))
for bn in range(2**num_qubits):
    target = format(bn, "0{:d}b".format(num_qubits))
    string_data = np.genfromtxt(file_address + 'Basis'+target+'.csv', delimiter=',',dtype=np.str)
    mean_mat[:,int(target,2)] = dictToVec(num_qubits,dict(Counter(string_data)))/string_data.shape[0]

In [ ]:
plt.rcParams["figure.figsize"] = (10, 11)
plt.rcParams["figure.dpi"] = 100

print_info(mean_mat)
plt.imshow(mean_mat)
plt.colorbar()
plt.title("Full Matrix")
plt.show()

In [ ]:
print_info(ind_mat)
plt.imshow(ind_mat)
plt.colorbar()
plt.title("Ind Matrix")
plt.show()

In [ ]:
# Differences
diff_mat = mean_mat - ind_mat
print_info(diff_mat)

plt.imshow(diff_mat)
plt.colorbar()
plt.title("Full - Ind")
plt.show()

In [ ]:
test_mat = np.copy(diff_mat)
test_mat[test_mat >= 0] = 1
test_mat[test_mat < 0] = 0

plt.imshow(test_mat)
plt.colorbar()
plt.title("0-1 -+")
plt.show()

In [ ]:
test_mat = np.copy(mean_mat)
test_mat[test_mat < 1e-6] = 0
test_mat[test_mat >= 1e-6] = 1

plt.imshow(test_mat)
plt.colorbar()
plt.title("0-1 1e-6")
plt.show()

In [ ]:
test_mat = np.copy(ind_mat)
test_mat[test_mat < 1e-6] = 0
test_mat[test_mat >= 1e-6] = 1

plt.imshow(test_mat)
plt.colorbar()
plt.title("0-1 1e-6")
plt.show()

In [ ]:
test_mat = np.copy(np.abs(diff_mat))
test_mat[test_mat < 1e-5] = 0
test_mat[test_mat >= 1e-5] = 1

plt.imshow(test_mat)
plt.colorbar()
plt.title("0-1 1e-5")
plt.show()

In [ ]:
test_mat = np.copy(np.abs(diff_mat))
test_mat[test_mat < 1e-6] = 0
test_mat[test_mat >= 1e-6] = 1

plt.imshow(test_mat)
plt.colorbar()
plt.title("0-1 1e-6")
plt.show()